# Feature Extraction

In [26]:
pip install category_encoders

In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gc

import category_encoders as ce

root = "C:\\Users\\hp\\OneDrive\\Desktop\\Instacart_Market_Analysis_Project_files\\"

In [28]:
aisles = pd.read_csv(root + 'aisles.csv')

departments = pd.read_csv(root + 'departments.csv')

orders = pd.read_csv(root + 'orders.csv', 
                 dtype={
                        'order_id': np.int32,
                        'user_id': np.int64,
                        'eval_set': 'category',
                        'order_number': np.int16,
                        'order_dow': np.int8,
                        'order_hour_of_day': np.int8,
                        'days_since_prior_order': np.float32})

order_products_prior = pd.read_csv(root + 'order_products__prior.csv', 
                                 dtype={
                                        'order_id': np.int32,
                                        'product_id': np.uint16,
                                        'add_to_cart_order': np.int16,
                                        'reordered': np.int8})

order_products_train = pd.read_csv(root + 'order_products__train.csv', 
                                 dtype={
                                        'order_id': np.int32,
                                        'product_id': np.uint16,
                                        'add_to_cart_order': np.int16,
                                        'reordered': np.int8})
products = pd.read_csv(root + 'products.csv')

### Preparing Data

In [29]:
prior_df = order_products_prior.merge(orders, on ='order_id', how='inner')
prior_df = prior_df.merge(products, on = 'product_id', how = 'left')
prior_df.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id
0,2,33120,1,1,202279,prior,3,5,9,8.0,Organic Egg Whites,86,16
1,2,28985,2,1,202279,prior,3,5,9,8.0,Michigan Organic Kale,83,4
2,2,9327,3,0,202279,prior,3,5,9,8.0,Garlic Powder,104,13
3,2,45918,4,1,202279,prior,3,5,9,8.0,Coconut Butter,19,13
4,2,30035,5,0,202279,prior,3,5,9,8.0,Natural Sweetener,17,13


## Features creation

Calculating how many times a user buy the product

In [30]:
prior_df['user_buy_product_times'] = prior_df.groupby(['user_id', 'product_id']).cumcount() + 1
prior_df.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,user_buy_product_times
0,2,33120,1,1,202279,prior,3,5,9,8.0,Organic Egg Whites,86,16,1
1,2,28985,2,1,202279,prior,3,5,9,8.0,Michigan Organic Kale,83,4,1
2,2,9327,3,0,202279,prior,3,5,9,8.0,Garlic Powder,104,13,1
3,2,45918,4,1,202279,prior,3,5,9,8.0,Coconut Butter,19,13,1
4,2,30035,5,0,202279,prior,3,5,9,8.0,Natural Sweetener,17,13,1


### Product level features 

(1) Product's average add-to-cart-order

(2) Total times the product was ordered

(3) Total times the product was reordered

(4) Reorder percentage of a product

(5) Total unique users of a product

(6) Is the product Organic?

(7) Percentage of users that buy the product second time

In [31]:
prod_feats1 = prior_df.groupby('product_id').agg(
    mean_add_to_cart_order=('add_to_cart_order', 'mean'),
    total_orders=('reordered', 'count'),
    total_reorders=('reordered', 'sum'),
    reorder_percentage=('reordered', 'mean'),
    unique_users=('user_id', lambda x: x.nunique()),
    order_first_time_total_cnt=('user_buy_product_times', lambda x: sum(x == 1)),
    order_second_time_total_cnt=('user_buy_product_times', lambda x: sum(x == 2)),
    is_organic=('product_name', lambda x: 1 if 'Organic' in x else 0)
).reset_index()

prod_feats1.head()

,product_id,mean_add_to_cart_order,total_orders,total_reorders,reorder_percentage,unique_users,order_first_time_total_cnt,order_second_time_total_cnt,is_organic
0,1,5.801836,1852,1136,0.613391,716,716,276,0
1,2,9.888889,90,12,0.133333,78,78,8,0
2,3,6.415162,277,203,0.732852,74,74,36,0
3,4,9.507599,329,147,0.446809,182,182,64,0
4,5,6.466667,15,9,0.600000,6,6,4,0


### Aisle and department features

(8) Reorder percentage, Total orders and reorders of a product  aisle

(9) Mean and std of aisle add-to-cart-order

(10) Aisle unique users

**features**

(10) Reorder percentage, Total orders and reorders of a product department

(11) Mean and std of department add-to-cart-order

(12) Department unique users

In [32]:
aisle_feats = prior_df.groupby('aisle_id').agg(
    aisle_mean_add_to_cart_order=('add_to_cart_order', 'mean'),
    aisle_std_add_to_cart_order=('add_to_cart_order', 'std'),
    aisle_total_orders=('reordered', 'count'),
    aisle_total_reorders=('reordered', 'sum'),
    aisle_reorder_percentage=('reordered', 'mean'),
    aisle_unique_users=('user_id', lambda x: x.nunique())
).reset_index()

aisle_feats.head()

,aisle_id,aisle_mean_add_to_cart_order,aisle_std_add_to_cart_order,aisle_total_orders,aisle_total_reorders,aisle_reorder_percentage,aisle_unique_users
0,1,8.167640,7.104166,71928,42912,0.596597,20711
1,2,9.275497,7.473802,82491,40365,0.489326,31222
2,3,9.571935,7.899672,456386,272922,0.598007,63592
3,4,10.161450,7.745705,200687,98243,0.489533,53892
4,5,10.297600,8.187047,62510,17542,0.280627,32312


In [33]:
dpt_feats = prior_df.groupby('department_id').agg(
    department_mean_add_to_cart_order=('add_to_cart_order', 'mean'),
    department_std_add_to_cart_order=('add_to_cart_order', 'std'),
    department_total_orders=('reordered', 'count'),
    department_total_reorders=('reordered', 'sum'),
    department_reorder_percentage=('reordered', 'mean'),
    department_unique_users=('user_id', lambda x: x.nunique())
).reset_index()

dpt_feats.head()

,department_id,department_mean_add_to_cart_order,department_std_add_to_cart_order,department_total_orders,department_total_reorders,department_reorder_percentage,department_unique_users
0,1,8.996414,7.393502,2236432,1211890,0.541885,163233
1,2,8.277645,7.526272,36291,14806,0.407980,17875
2,3,8.084397,6.904849,1176787,739188,0.628141,140612
3,4,8.022875,6.658899,9479291,6160710,0.649913,193237
4,5,5.428346,5.778253,153696,87595,0.569924,15798


**features**

(13) Binary encoding of aisle feature

(14) Binary encoding of department feature

In [34]:
prod_feats1 = prod_feats1.merge(products, on = 'product_id', how = 'left')
prod_feats1 = prod_feats1.merge(aisle_feats, on = 'aisle_id', how = 'left')
prod_feats1 = prod_feats1.merge(aisles, on = 'aisle_id', how = 'left')
prod_feats1 = prod_feats1.merge(dpt_feats, on = 'department_id', how = 'left')
prod_feats1 = prod_feats1.merge(departments, on = 'department_id', how = 'left')
prod_feats1.head()

,product_id,mean_add_to_cart_order,total_orders,total_reorders,reorder_percentage,unique_users,order_first_time_total_cnt,order_second_time_total_cnt,is_organic,product_name,...,aisle_reorder_percentage,aisle_unique_users,aisle,department_mean_add_to_cart_order,department_std_add_to_cart_order,department_total_orders,department_total_reorders,department_reorder_percentage,department_unique_users,department
0,1,5.801836,1852,1136,0.613391,716,716,276,0,Chocolate Sandwich Cookies,...,0.548698,54202,cookies cakes,9.187743,7.692492,2887550,1657973,0.574180,174219,snacks
1,2,9.888889,90,12,0.133333,78,78,8,0,All-Seasons Salt,...,0.152391,76402,spices seasonings,9.593425,7.875241,1875577,650301,0.346721,172755,pantry
2,3,6.415162,277,203,0.732852,74,74,36,0,Robust Golden Unsweetened Oolong Tea,...,0.527615,53197,tea,6.976699,6.711172,2690129,1757892,0.653460,172795,beverages
3,4,9.507599,329,147,0.446809,182,182,64,0,Smart Ones Classic Favorites Mini Rigatoni Wit...,...,0.556655,58749,frozen meals,8.996414,7.393502,2236432,1211890,0.541885,163233,frozen
4,5,6.466667,15,9,0.600000,6,6,4,0,Green Chile Anytime Sauce,...,0.280627,32312,marinades meat preparation,9.593425,7.875241,1875577,650301,0.346721,172755,pantry


In [35]:
prod_feats1.drop(['product_name', 'aisle_id', 'department_id'], axis = 1, inplace = True)
prod_feats1.head()

,product_id,mean_add_to_cart_order,total_orders,total_reorders,reorder_percentage,unique_users,order_first_time_total_cnt,order_second_time_total_cnt,is_organic,aisle_mean_add_to_cart_order,...,aisle_reorder_percentage,aisle_unique_users,aisle,department_mean_add_to_cart_order,department_std_add_to_cart_order,department_total_orders,department_total_reorders,department_reorder_percentage,department_unique_users,department
0,1,5.801836,1852,1136,0.613391,716,716,276,0,9.253092,...,0.548698,54202,cookies cakes,9.187743,7.692492,2887550,1657973,0.574180,174219,snacks
1,2,9.888889,90,12,0.133333,78,78,8,0,9.996181,...,0.152391,76402,spices seasonings,9.593425,7.875241,1875577,650301,0.346721,172755,pantry
2,3,6.415162,277,203,0.732852,74,74,36,0,8.519846,...,0.527615,53197,tea,6.976699,6.711172,2690129,1757892,0.653460,172795,beverages
3,4,9.507599,329,147,0.446809,182,182,64,0,9.207741,...,0.556655,58749,frozen meals,8.996414,7.393502,2236432,1211890,0.541885,163233,frozen
4,5,6.466667,15,9,0.600000,6,6,4,0,10.297600,...,0.280627,32312,marinades meat preparation,9.593425,7.875241,1875577,650301,0.346721,172755,pantry


In [36]:
prod_feats1.shape

(49677, 23)

In [37]:
prod_feats1.dtypes

product_id                            uint16
mean_add_to_cart_order               float64
total_orders                           int64
total_reorders                         int64
reorder_percentage                   float64
unique_users                           int64
order_first_time_total_cnt             int64
order_second_time_total_cnt            int64
is_organic                             int64
aisle_mean_add_to_cart_order         float64
aisle_std_add_to_cart_order          float64
aisle_total_orders                     int64
aisle_total_reorders                   int64
aisle_reorder_percentage             float64
aisle_unique_users                     int64
aisle                                 object
department_mean_add_to_cart_order    float64
department_std_add_to_cart_order     float64
department_total_orders                int64
department_total_reorders              int64
department_reorder_percentage        float64
department_unique_users                int64
department

In [38]:
encoder= ce.BinaryEncoder(cols=['aisle', 'department'],return_df=True)

In [39]:
prod_feats1 = encoder.fit_transform(prod_feats1)
prod_feats1.head()

,product_id,mean_add_to_cart_order,total_orders,total_reorders,reorder_percentage,unique_users,order_first_time_total_cnt,order_second_time_total_cnt,is_organic,aisle_mean_add_to_cart_order,...,department_std_add_to_cart_order,department_total_orders,department_total_reorders,department_reorder_percentage,department_unique_users,department_0,department_1,department_2,department_3,department_4
0,1,5.801836,1852,1136,0.613391,716,716,276,0,9.253092,...,7.692492,2887550,1657973,0.574180,174219,0,0,0,0,1
1,2,9.888889,90,12,0.133333,78,78,8,0,9.996181,...,7.875241,1875577,650301,0.346721,172755,0,0,0,1,0
2,3,6.415162,277,203,0.732852,74,74,36,0,8.519846,...,6.711172,2690129,1757892,0.653460,172795,0,0,0,1,1
3,4,9.507599,329,147,0.446809,182,182,64,0,9.207741,...,7.393502,2236432,1211890,0.541885,163233,0,0,1,0,0
4,5,6.466667,15,9,0.600000,6,6,4,0,10.297600,...,7.875241,1875577,650301,0.346721,172755,0,0,0,1,0


In [40]:
prod_feats1.shape

(49677, 34)

In [41]:
prod_feats1.columns

Index(['product_id', 'mean_add_to_cart_order', 'total_orders',
       'total_reorders', 'reorder_percentage', 'unique_users',
       'order_first_time_total_cnt', 'order_second_time_total_cnt',
       'is_organic', 'aisle_mean_add_to_cart_order',
       'aisle_std_add_to_cart_order', 'aisle_total_orders',
       'aisle_total_reorders', 'aisle_reorder_percentage',
       'aisle_unique_users', 'aisle_0', 'aisle_1', 'aisle_2', 'aisle_3',
       'aisle_4', 'aisle_5', 'aisle_6', 'aisle_7',
       'department_mean_add_to_cart_order', 'department_std_add_to_cart_order',
       'department_total_orders', 'department_total_reorders',
       'department_reorder_percentage', 'department_unique_users',
       'department_0', 'department_1', 'department_2', 'department_3',
       'department_4'],
      dtype='object')

In [42]:
prod_feats1.isnull().any().any()

np.False_

In [ ]:
# free some memory
del aisle_feats, dpt_feats, aisles, departments
gc.collect()

### User level features

(15) User's average and std day-of-week of order

(16) User's average and std hour-of-day of order

(17) User's average and std days-since-prior-order

(18) Total orders by a user

(19) Total products user has bought

(20) Total unique products user has bought

(21) user's total reordered products

(22) User's overall reorder percentage

In [44]:
prior_df.isnull().any()

order_id                  False
product_id                False
add_to_cart_order         False
reordered                 False
user_id                   False
eval_set                  False
order_number              False
order_dow                 False
order_hour_of_day         False
days_since_prior_order     True
product_name              False
aisle_id                  False
department_id             False
user_buy_product_times    False
dtype: bool

In [45]:
# when no prior order, the value is null. Imputing as 0
prior_df.days_since_prior_order = prior_df.days_since_prior_order.fillna(0)

In [46]:
prior_df.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,user_buy_product_times
0,2,33120,1,1,202279,prior,3,5,9,8.0,Organic Egg Whites,86,16,1
1,2,28985,2,1,202279,prior,3,5,9,8.0,Michigan Organic Kale,83,4,1
2,2,9327,3,0,202279,prior,3,5,9,8.0,Garlic Powder,104,13,1
3,2,45918,4,1,202279,prior,3,5,9,8.0,Coconut Butter,19,13,1
4,2,30035,5,0,202279,prior,3,5,9,8.0,Natural Sweetener,17,13,1


In [47]:
# Define the aggregation dictionary without nested renaming
agg_dict4 = {
    'order_dow': ['mean', 'std'], 
    'order_hour_of_day': ['mean', 'std'],
    'days_since_prior_order': ['mean', 'std'],
    'order_number': [lambda x: x.nunique()],
    'product_id': ['count', lambda x: x.nunique()],
    'reordered': ['sum', 'mean']
}

# Group by 'user_id' and apply the aggregation functions
user_feats = prior_df.groupby('user_id').agg(agg_dict4)

# Manually rename the columns after aggregation
user_feats.columns = [
    'avg_dow', 'std_dow',
    'avg_doh', 'std_doh',
    'avg_since_order', 'std_since_order',
    'total_orders_by_user', 
    'total_products_by_user', 'total_unique_product_by_user',
    'total_reorders_by_user', 'reorder_propotion_by_user'
]

# Reset the index to include 'user_id' as a normal column
user_feats.reset_index(inplace=True)

# Display the result
user_feats.head()

,user_id,avg_dow,std_dow,avg_doh,std_doh,avg_since_order,std_since_order,total_orders_by_user,total_products_by_user,total_unique_product_by_user,total_reorders_by_user,reorder_propotion_by_user
0,1,2.644068,1.256194,10.542373,3.500355,18.542374,10.559066,10,59,18,41,0.694915
1,2,2.005128,0.971222,10.441026,1.649854,14.902564,9.671712,14,195,102,93,0.476923
2,3,1.011364,1.245630,16.352273,1.454599,10.181818,5.867396,12,88,33,55,0.625000
3,4,4.722222,0.826442,13.111111,1.745208,11.944445,9.973330,5,18,17,1,0.055556
4,5,1.621622,1.276961,15.729730,2.588958,10.189189,7.600577,4,37,23,14,0.378378


**features**

(23) Average order size of a user

(24) User's mean of reordered items of all orders

In [48]:
# Define the aggregation dictionary without nested renaming
agg_dict5 = {
    'reordered': ['count', 'mean']
}

# Group by 'user_id' and 'order_number' and apply the aggregation functions
user_feats2 = prior_df.groupby(['user_id', 'order_number']).agg(agg_dict5)

# Manually rename the columns after aggregation
user_feats2.columns = ['average_order_size', 'reorder_in_order']

# Reset the index to include 'user_id' and 'order_number' as normal columns
user_feats2.reset_index(inplace=True)

# Display the result
user_feats2.head()

,user_id,order_number,average_order_size,reorder_in_order
0,1,1,5,0.000
1,1,2,6,0.500
2,1,3,5,0.600
3,1,4,5,1.000
4,1,5,8,0.625


In [49]:
# Group by 'user_id' and calculate the mean of 'average_order_size' and 'reorder_in_order'
user_feats3 = user_feats2.groupby('user_id').agg({
    'average_order_size': 'mean',
    'reorder_in_order': 'mean'
})

# Reset the index to include 'user_id' as a normal column
user_feats3 = user_feats3.reset_index()

# Display the result
user_feats3.head()


,user_id,average_order_size,reorder_in_order
0,1,5.900000,0.705833
1,2,13.928571,0.447961
2,3,7.333333,0.658817
3,4,3.600000,0.028571
4,5,9.250000,0.377778


In [50]:
user_feats = user_feats.merge(user_feats3, on = 'user_id', how = 'left')
user_feats.head()

,user_id,avg_dow,std_dow,avg_doh,std_doh,avg_since_order,std_since_order,total_orders_by_user,total_products_by_user,total_unique_product_by_user,total_reorders_by_user,reorder_propotion_by_user,average_order_size,reorder_in_order
0,1,2.644068,1.256194,10.542373,3.500355,18.542374,10.559066,10,59,18,41,0.694915,5.900000,0.705833
1,2,2.005128,0.971222,10.441026,1.649854,14.902564,9.671712,14,195,102,93,0.476923,13.928571,0.447961
2,3,1.011364,1.245630,16.352273,1.454599,10.181818,5.867396,12,88,33,55,0.625000,7.333333,0.658817
3,4,4.722222,0.826442,13.111111,1.745208,11.944445,9.973330,5,18,17,1,0.055556,3.600000,0.028571
4,5,1.621622,1.276961,15.729730,2.588958,10.189189,7.600577,4,37,23,14,0.378378,9.250000,0.377778


**features**

(25) Percentage of reordered itmes in user's last three orders

(26) Total orders in user's last three orders

Last 3 orders of a user

In [51]:
# Sort the data by 'user_id' and 'order_number' in descending order
user_feats2_sorted = user_feats2.sort_values(by=['user_id', 'order_number'], ascending=[True, False])

# Group by 'user_id' and select the last 3 orders for each user
last_three_orders = user_feats2_sorted.groupby('user_id').head(3).reset_index(drop=True)

# Display the result
last_three_orders.head()

,user_id,order_number,average_order_size,reorder_in_order
0,1,10,9,0.666667
1,1,9,6,1.000000
2,1,8,6,0.666667
3,2,14,16,0.625000
4,2,13,9,0.000000


In [52]:
# Sort the data by 'user_id' and 'order_number' in descending order
user_feats2_sorted = user_feats2.sort_values(by=['user_id', 'order_number'], ascending=[True, False])

# Group by 'user_id' and select the last 3 orders for each user
last_three_orders = user_feats2_sorted.groupby('user_id').head(3).reset_index(drop=True)

# Merge 'user_feats2' with 'last_three_orders' to retain the rows for the last 3 orders
last_three_orders_merged = user_feats2.merge(last_three_orders, on=['user_id', 'order_number'], how='inner')

# Display the merged result
last_three_orders_merged.head()


,user_id,order_number,average_order_size_x,reorder_in_order_x,average_order_size_y,reorder_in_order_y
0,1,8,6,0.666667,6,0.666667
1,1,9,6,1.000000,6,1.000000
2,1,10,9,0.666667,9,0.666667
3,2,12,19,0.578947,19,0.578947
4,2,13,9,0.000000,9,0.000000


In [53]:
last_three_orders['rank'] = last_three_orders.groupby("user_id")["order_number"].rank("dense", ascending=True)

In [54]:
last_order_feats = last_three_orders.pivot_table(index = 'user_id', columns = ['rank'], \
                                                 values=['average_order_size', 'reorder_in_order']).\
                                                reset_index(drop = False)
last_order_feats.columns = ['user_id','orders_3', 'orders_2', 'orders_1', 'reorder_3', 'reorder_2', 'reorder_1']
last_order_feats.head()

,user_id,orders_3,orders_2,orders_1,reorder_3,reorder_2,reorder_1
0,1,6.0,6.0,9.0,0.666667,1.0,0.666667
1,2,19.0,9.0,16.0,0.578947,0.0,0.625000
2,3,6.0,5.0,6.0,0.833333,1.0,1.000000
3,4,7.0,2.0,3.0,0.142857,0.0,0.000000
4,5,9.0,5.0,12.0,0.444444,0.4,0.666667


In [55]:
user_feats = user_feats.merge(last_order_feats, on = 'user_id', how = 'left')
user_feats.head()

,user_id,avg_dow,std_dow,avg_doh,std_doh,avg_since_order,std_since_order,total_orders_by_user,total_products_by_user,total_unique_product_by_user,total_reorders_by_user,reorder_propotion_by_user,average_order_size,reorder_in_order,orders_3,orders_2,orders_1,reorder_3,reorder_2,reorder_1
0,1,2.644068,1.256194,10.542373,3.500355,18.542374,10.559066,10,59,18,41,0.694915,5.900000,0.705833,6.0,6.0,9.0,0.666667,1.0,0.666667
1,2,2.005128,0.971222,10.441026,1.649854,14.902564,9.671712,14,195,102,93,0.476923,13.928571,0.447961,19.0,9.0,16.0,0.578947,0.0,0.625000
2,3,1.011364,1.245630,16.352273,1.454599,10.181818,5.867396,12,88,33,55,0.625000,7.333333,0.658817,6.0,5.0,6.0,0.833333,1.0,1.000000
3,4,4.722222,0.826442,13.111111,1.745208,11.944445,9.973330,5,18,17,1,0.055556,3.600000,0.028571,7.0,2.0,3.0,0.142857,0.0,0.000000
4,5,1.621622,1.276961,15.729730,2.588958,10.189189,7.600577,4,37,23,14,0.378378,9.250000,0.377778,9.0,5.0,12.0,0.444444,0.4,0.666667


### User and Product level features

(27) User's avg add-to-cart-order for a product

(28) User's avg days_since_prior_order for a product

(29) User's product total orders, reorders and reorders percentage

(30) User's order number when the product was bought last

In [ ]:
# Define aggregation dictionary properly
agg_dict6 = {
    'add_to_cart_order': ['mean', 'min', 'max', 'std'],  
    'reordered': ['sum', 'count', 'mean'],  
    'order_number': ['min', 'max', 'count']
}

# Group by 'user_id' and 'product_id', and apply aggregations
user_product_feats = prior_df.groupby(['user_id', 'product_id']).agg(agg_dict6)

# Flatten multi-index column names (if needed)
user_product_feats.columns = ['_'.join(col) for col in user_product_feats.columns]

# Reset index to keep 'user_id' and 'product_id' as columns
user_product_feats.reset_index(inplace=True)

# Display the processed data
user_product_feats.head()

**features**

(31) User's product purchase history of last three orders

In [ ]:
last_three_orders.head()

In [ ]:
last_orders = prior_df.merge(last_three_orders, on = ['user_id', 'order_number'], how = 'inner')
last_orders.head()

In [ ]:
last_orders['rank'] = last_orders.groupby(['user_id', 'product_id'])['order_number'].rank("dense", ascending=True)

In [ ]:
product_purchase_history = last_orders.pivot_table(index = ['user_id', 'product_id'],\
                                                   columns='rank', values = 'reordered').reset_index()
product_purchase_history.columns = ['user_id', 'product_id', 'is_reorder_3', 'is_reorder_2', 'is_reorder_1']
product_purchase_history.fillna(0, inplace = True)
product_purchase_history.head()

In [ ]:
user_product_feats = user_product_feats.merge(product_purchase_history, on=['user_id', 'product_id'], how = 'left')
user_product_feats.head()

In [ ]:
user_product_feats.isnull().sum()

In [ ]:
user_product_feats.fillna(0, inplace = True)

## Saving all features

In [ ]:
prod_feats1.to_pickle(root + 'product_features.pkl')
user_feats.to_pickle(root +'user_features.pkl')
user_product_feats.to_pickle(root +'user_product_features.pkl')

In [ ]:
df = pd.read_pickle(root +'product_features.pkl')
df.head()

In [ ]:
df = pd.read_pickle(root+'user_features.pkl')
df.head()

In [ ]:
df = pd.read_pickle(root + 'user_product_features.pkl')
df.head()